<a href="https://colab.research.google.com/github/ShimilSBabu/Facial_Expression_Detector/blob/main/facial_expression_detection_functional_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/ahmedmoorsy/facial-expression")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shimilsbabu
Your Kaggle Key: ··········


100%|██████████| 193M/193M [00:06<00:00, 30.3MB/s]

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../content/facial-expression/fer2013/"))

# Any results you write to the current directory are saved as output.

['fer2013.csv']


In [14]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K

In [5]:
# get the data
filname = '../content/facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('../content/facial-expression/fer2013/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [6]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y


In [7]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

7


In [8]:
# keras with tensorflow backend
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [29]:
from keras.models import Sequential
from tensorflow.keras.models import Model
from keras.layers import Dense , Activation , Dropout ,Flatten, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers import BatchNormalization
from tensorflow.keras.utils import load_img, img_to_array

In [40]:
def custom_functional_model():
  input = Input((48,48,1))
  conv_block_1 = Conv2D(64, (5, 5), activation='relu', padding='same')(input)
  conv_block_1 = Conv2D(64, (5, 5), activation='relu', padding='same')(conv_block_1)
  conv_block_1 = BatchNormalization()(conv_block_1)
  # conv_block_1 = MaxPooling2D(pool_size=(2, 2))(conv_block_1)
  conv_block_1 = Dropout(0.3)(conv_block_1)


  conv_block_2 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv_block_1)
  conv_block_2 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv_block_2)
  conv_block_2 = MaxPooling2D(pool_size=(2, 2))(conv_block_2)
  conv_block_2 = Dropout(0.3)(conv_block_2)

  conv_block_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_block_2)
  conv_block_3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_block_3)
  # conv_block_3 = BatchNormalization()(conv_block_3)
  conv_block_3 = MaxPooling2D(pool_size=(2, 2))(conv_block_3)
  conv_block_3 = Dropout(0.3)(conv_block_3)

  conv_block_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_block_3)
  conv_block_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_block_4)
  # conv_block_3 = BatchNormalization()(conv_block_3)
  conv_block_4 = MaxPooling2D(pool_size=(2, 2))(conv_block_4)
  conv_block_4 = Dropout(0.3)(conv_block_4)

  # output_block = Flatten()(conv_block_3)
  output_block = GlobalAveragePooling2D()(conv_block_4)

  output_block = Dense(64)(output_block)
  # output_block = BatchNormalization()(output_block)
  output_block = Activation('relu')(output_block)
  output_block = Dropout(0.2)(output_block)
  output_block = Dense(7)(output_block)
  output_block = Activation('softmax')(output_block)

  model = Model(input, output_block)
  
  return model 

In [41]:
model=custom_functional_model()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 48, 48, 64)        1664      
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 48, 48, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 48, 48, 128)       204928    
                                                           

In [42]:
path_model='model_filter.h5' # save model at this location after each epoch
K.clear_session() # destroys the current graph and builds a new one
# model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate

In [43]:
# fit the model
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=50, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

Epoch 1/50
505/505 [==============================] - 109s 103ms/step - loss: 1.8141 - accuracy: 0.2402 - val_loss: 1.8252 - val_accuracy: 0.2563
Epoch 2/50
505/505 [==============================] - 51s 101ms/step - loss: 1.7899 - accuracy: 0.2506 - val_loss: 1.7647 - val_accuracy: 0.2825
Epoch 3/50
505/505 [==============================] - 51s 101ms/step - loss: 1.7618 - accuracy: 0.2727 - val_loss: 1.8554 - val_accuracy: 0.2343
Epoch 4/50
505/505 [==============================] - 51s 101ms/step - loss: 1.7168 - accuracy: 0.2965 - val_loss: 1.7381 - val_accuracy: 0.3031
Epoch 5/50
505/505 [==============================] - 52s 103ms/step - loss: 1.6276 - accuracy: 0.3530 - val_loss: 1.5996 - val_accuracy: 0.3706
Epoch 6/50
505/505 [==============================] - 51s 101ms/step - loss: 1.5177 - accuracy: 0.4139 - val_loss: 1.6376 - val_accuracy: 0.3700
Epoch 7/50
505/505 [==============================] - 52s 102ms/step - loss: 1.4390 - accuracy: 0.4457 - val_loss: 1.4508 - val_a

In [44]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

[0 1 2 3 4 5 6]


In [45]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [46]:
y_pred=model.predict(X_test)
#print(y_pred)
y_test.shape

113/113 [==============================] - 3s 17ms/step


(3589, 7)

In [47]:
#import seaborn as sn
#import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
#from sklearn.metrics import confusion_matrix
#%matplotlib inline
#cm = confusion_matrix(np.where(y_test == 1)[1], y_pred)
#cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#df_cm = pd.DataFrame(cm, index = [i for i in "0123456"],
                  #columns = [i for i in "0123456"])
#plt.figure(figsize = (20,15))
#sn.heatmap(df_cm, annot=True)

**Real Time Expression Prediction**

In [ ]:
from skimage import io
image_path = input('Enter the image path : ')
# image_path = '/content/download.jpg'
img = load_img(image_path, grayscale=True, target_size=(48, 48))
show_img= load_img(image_path, grayscale=False, target_size=(200, 200))
x = img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
#print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])
        

**Live Demo of Production Level Project**

[Facial Expression Detection Web App](https://faceai.herokuapp.com/)